# SIT automatic behaviour detection pipeline
## By Santiago Holguin Urbano

This pipeline allows to identify different behaviours of two B6 mice. The detected behaviuors are : Contact by host, Contact by visitor, Follow by host, Follow by visitor,Paw control (host), Rearing (host), and Grooming (host).

This pipeline can be used with pretrained models on videos at the following format : 
[image-2.png](attachment:image-2.png)

But the different tools used can be adapated for different format and different behaviours. If you're working with CD1 mice or with a very different environment, we suggest you to retrain all the models by following each method documentation. You need at least 600 annotated frames for tracking, and at least 9k annotated frames per beahaviour for behaviuor estimation.

## 1. Tracking videos 
Make sur to be working on the project **ROOT**,  and to have copied your videos to `for_tracking` folder.

Then execute this code on your terminal (powersheell) : `.\track_all_videos.ps1`

The first block will produce a .slp file for the video, this file cannot be read alone, and it contains different tracking features. To combine this file with a video and correct possible tracking errors, you can use our GUI 
# 2. Correcting tracking errors (Red = resident) : sleap-no-GUI kernel

This block has added tracked videos to [corrected_videos](corrected_videos) folder

In [1]:
!conda run -n sleap_andGUI python correcting_sleap_errors.py

# 3. Add videos to deepEthogram : change kernel to Deg-ipynb or do it directly from the Deg GUi -- Recommended (see [deg documentation](https://github.com/jbohnslav/deepethogram/blob/master/docs/getting_started.md))

`conda activate deg_social`

`deepethogram`

In [ ]:
import os
from deepethogram import projects
import pandas as pd

def añadir_videos_sin_reparar(project_config_path, carpeta_videos, modo='copy', extensiones=None, csv_filtro_path=None):
    if extensiones is None:
        extensiones = ['.mp4', '.avi', '.mov', '.mkv', '.wmv']
    
    # Cargar proyecto
    proyecto = projects.load_config(project_config_path)
    
    # Cargar filtro CSV si se da
    nombres_permitidos = None
    if csv_filtro_path and os.path.exists(csv_filtro_path):
        df_filtro = pd.read_csv(csv_filtro_path)
        nombres_permitidos = set(os.path.splitext(v)[0] for v in df_filtro['video_name'].values)
        print(f"[🔎] Usando filtro CSV con {len(nombres_permitidos)} videos permitidos.")
    
    # Listar videos válidos
    videos_para_añadir = []
    for archivo in os.listdir(carpeta_videos):
        ruta_completa = os.path.join(carpeta_videos, archivo)
        if os.path.isfile(ruta_completa) and os.path.splitext(archivo)[1].lower() in extensiones:
            nombre_sin_ext = os.path.splitext(archivo)[0].replace(" ", "")
            if nombres_permitidos is None or nombre_sin_ext in nombres_permitidos:
                videos_para_añadir.append(ruta_completa)
    
    print(f"[🔍] Encontrados {len(videos_para_añadir)} videos para añadir.")
    
    # Añadir videos al proyecto
    cont = 0
    for video_path in videos_para_añadir:
        try:
            nueva_ruta = projects.add_video_to_project(proyecto, video_path, mode=modo)
            cont += 1
            print(f"[+] Añadido: {video_path} → {nueva_ruta} ({cont}/{len(videos_para_añadir)})")
        except Exception as e:
            print(f"[✗] Error añadiendo {video_path}: {e}")

# === CONFIGURACIÓN ===
config_path = r"D:\SIT_auto\SIT_deepethogram\project_config.yaml"
carpeta_videos = r"corrected_videos"

añadir_videos_sin_reparar(config_path, carpeta_videos=carpeta_videos, modo='copy', csv_filtro_path=None)


# 4. Adding annotations
You can either open and manually annotate videos in DeepEthogram, or you can transform solomon csv annotations on DeG compatible csv files by using this script : [transform](training_data_conversion\LBNtrasnformCSV.py)

# 5. Train DeG models (see [deg documentation](https://github.com/jbohnslav/deepethogram/blob/master/docs/getting_started.md))
Once you've annotate multiple videos you will be able to train a model